In [ ]:
from collections import Counter
from os.path import join
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.utils import *
from utils.get_data import load_data
from utils.get_generated_data import load_generated_data
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

# Gender classifier on non verbal behaviour


In [ ]:

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)
    

class ConvClassifier(nn.Module):

    def __init__(self, latent_dim, nb_classes):
        super().__init__()

        self.conv1 = Conv(latent_dim, 32)
        self.conv2 = Conv(32, 64)
        self.fc1 = torch.nn.Linear(64 * 25, 64)
        self.fc2 = torch.nn.Linear(64, nb_classes)

    def forward(self, x): #(512,6)
        x = x.swapaxes(1, 2)
        x = self.conv1(x)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = self.conv2(x)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:

def conf_matrix(y_test, y_pred):
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Matrice de confusion :\n")
    print(str(conf_matrix))
    errors_per_class = []
    for i in range(len(conf_matrix)):
        errors_per_class.append(sum(conf_matrix[i]) - conf_matrix[i][i])
    print("\nNombre d'erreurs par classe :\n")
    for i in range(len(errors_per_class)):
        print(f"Classe {i}: {errors_per_class[i]} erreurs")

def test_model(model, test_loader, criterion):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            item, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.squeeze().numpy())
            all_labels.extend(labels)
    all_labels = [one_hot_to_index(ele, "gender") for ele in all_labels]
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    return loss, accuracy, f1, all_labels, all_predictions

def train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        test_loss, test_accuracy, test_f1, all_labels, all_predictions = test_model(model, test_loader, criterion)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, test_loss: {test_loss:.4f}')
    print(f'\n*****Accuracy on test set: {test_accuracy:.4f}')
    print(f'*****F1 on test set: {test_f1:.4f}')
    conf_matrix(all_labels, all_predictions)
    return model, test_accuracy

In [ ]:
def supress_index(raw_labels_list, labels_list, x):
    indices_a_supprimer = []
    for i in range(len(raw_labels_list)):
        if "silence" in raw_labels_list[i]:
            indices_a_supprimer.append(i)
    tensor = x.clone()
    masque = torch.ones(x.size(0), dtype=torch.bool)
    masque[indices_a_supprimer] = False
    tensor_sans_indices = torch.index_select(tensor, dim=0, index=torch.nonzero(masque).squeeze())
    new_labels_list = torch.index_select(labels_list, dim=0, index=torch.nonzero(masque).squeeze())
    return tensor_sans_indices, new_labels_list

def reshape_for_classif(x_in, y_in, scaler=None):
    minMaxScaler = MinMaxScaler((-1,1))
    if scaler is None:
        scaler = minMaxScaler.fit(x_in.view(-1, x_in.size()[2])) 
    x_scaled = torch.empty(size=(x_in.size()[0], x_in.size()[1], x_in.size()[2]))
    for i in range(x_in.size()[0]):
        x_scaled[i] = torch.tensor(scaler.transform(x_in[i])) 
    return x_scaled, y_in, scaler

#### Load data

In [ ]:
dict = load_data(12)
dict["test_generated"] = load_generated_data("24-04-2024_trueness_1_CGAN_2/epoch_1250", create_init_files=True)

In [ ]:
visual_features = ["gaze_0_x", "gaze_0_y", "gaze_0_z", "gaze_1_x", "gaze_1_y", "gaze_1_z", "gaze_angle_x", "gaze_angle_y", "pose_Rx", "pose_Ry",
                "pose_Rz", "AU01_r", "AU02_r", "AU04_r", "AU05_r", "AU06_r", "AU07_r", "AU09_r", "AU10_r", "AU12_r", "AU14_r", "AU15_r", "AU17_r", "AU20_r", "AU23_r", "AU25_r", "AU26_r", "AU45_r"]

yeux = [visual_features.index("gaze_0_x"), visual_features.index("gaze_0_y"), visual_features.index("gaze_0_z"), visual_features.index("gaze_1_x"), visual_features.index("gaze_1_y"), visual_features.index("gaze_1_z")]
pose = [visual_features.index("pose_Rx"), visual_features.index("pose_Ry"), visual_features.index("pose_Rz")]
au = [visual_features.index("AU01_r"), visual_features.index("AU02_r"), visual_features.index("AU04_r"), visual_features.index("AU05_r"), visual_features.index("AU06_r"), visual_features.index("AU07_r"), visual_features.index("AU09_r"), visual_features.index("AU10_r"), visual_features.index("AU12_r"), visual_features.index("AU14_r"), visual_features.index("AU15_r"), visual_features.index("AU17_r"), visual_features.index("AU20_r"), visual_features.index("AU23_r"), visual_features.index("AU25_r"), visual_features.index("AU26_r"), visual_features.index("AU45_r")]
sourcils = [visual_features.index("AU01_r"), visual_features.index("AU02_r"), visual_features.index("AU04_r")]
visage = [visual_features.index("AU05_r"), visual_features.index("AU06_r"), visual_features.index("AU07_r"), visual_features.index("AU09_r"), visual_features.index("AU10_r")]
bouche = [visual_features.index("AU12_r"), visual_features.index("AU14_r"), visual_features.index("AU15_r"), visual_features.index("AU17_r"), visual_features.index("AU20_r"), visual_features.index("AU23_r"), visual_features.index("AU25_r"), visual_features.index("AU26_r")]
clignement = [visual_features.index("AU45_r")]

feature_index = [visual_features.index("AU06_r"), visual_features.index("AU07_r"), visual_features.index("AU14_r"), visual_features.index("AU15_r"),visual_features.index("AU20_r"),visual_features.index("AU26_r"), visual_features.index("AU45_r")]

#### With training

In [ ]:
label = "gender"
tensor = "one_hot_tensor_gender"
nb_labels = 3


train_data, train_labels = supress_index(dict["train"][label], dict["train"][tensor], dict["train"]["Y_behaviour"][:,:,:]) #features_index
test_data, test_labels = supress_index(dict["test"][label], dict["test"][tensor], dict["test"]["Y_behaviour"][:,:,:]) #features_index

X_train, y_train, scaler = reshape_for_classif(train_data, train_labels)
X_test, y_test, _ = reshape_for_classif(test_data, test_labels, scaler)


train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

classifier = ConvClassifier(train_data.shape[2], nb_labels)
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


for i in range(10):
    model, acc = train_model(classifier, criterion, optimizer, train_loader, test_loader, num_epochs=10)
    print("Test accuracy: ", acc)

#### Without training (using the saved model)

In [ ]:
dict = load_data(12)
dict["test_generated"] = load_generated_data("24-04-2024_trueness_1_CGAN_2/epoch_1250", create_init_files=True)

In [ ]:
label = "gender"
tensor = "one_hot_tensor_gender"
nb_labels = 3
test = "test_generated"

train_data, train_labels = supress_index(dict["train"][label], dict["train"][tensor], dict["train"]["Y_behaviour"][:,:,:])
test_data, test_labels = supress_index(dict[test][label], dict[test][tensor], dict[test]["Y_behaviour"][:,:,:])
X_train, y_train, scaler = reshape_for_classif(train_data, train_labels)
X_test, y_test, _ = reshape_for_classif(test_data, test_labels, scaler)

classifier = ConvClassifier(train_data.shape[2], nb_labels)
classifier.load_state_dict(torch.load("saved_models/gender_classifier.pt"))

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)


with torch.no_grad():
        classifier.eval()
        test_loss, test_accuracy, test_f1, all_labels, all_predictions = test_model(classifier, test_loader, nn.CrossEntropyLoss())
        print("Accuracy: ", test_accuracy)


# Gender classifier on audio

In [ ]:

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.Dropout(0.2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)
    

class ConvClassifier(nn.Module):

    def __init__(self, latent_dim, nb_classes):
        super().__init__()
        self.conv1 = Conv(latent_dim, 512)
        self.conv2 = Conv(512, 128)
        self.conv3 = Conv(128, 64)
        self.fc1 = torch.nn.Linear(64 * 25, 64)
        self.fc2 = torch.nn.Linear(64, nb_classes)

    def forward(self, x):
        x = x.swapaxes(1, 2)
        x = self.conv1(x)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = self.conv2(x)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = self.conv3(x)
        x = F.max_pool1d(x, kernel_size=2, stride=2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
def conf_matrix(y_test, y_pred):
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Matrice de confusion :\n")
    print(str(conf_matrix))
    errors_per_class = []
    for i in range(len(conf_matrix)):
        errors_per_class.append(sum(conf_matrix[i]) - conf_matrix[i][i])
    print("\nNombre d'erreurs par classe :\n")
    for i in range(len(errors_per_class)):
        print(f"Classe {i}: {errors_per_class[i]} erreurs")

def test_model(model, test_loader, criterion):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            item, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.squeeze().numpy())
            all_labels.extend(labels)
    all_labels = [one_hot_to_index(ele, label) for ele in all_labels]
    accuracy = accuracy_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    return loss, accuracy, f1, all_labels, all_predictions


def train_model(model, criterion, optimizer, train_loader, test_loader, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            #print(outputs, labels)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        test_loss, test_accuracy, test_f1, all_labels, all_predictions = test_model(model, test_loader, criterion)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, test_loss: {test_loss:.4f}, test_accuracy: {test_accuracy:.4f}')
    return model


In [ ]:
def reshape_for_classif(x_in, y_in, scaler=None):
    minMaxScaler = MinMaxScaler((0,1))
    if scaler is None:
        scaler = minMaxScaler.fit(x_in.view(-1, x_in.size()[2])) 
    x_scaled = torch.empty(size=(x_in.size()[0], x_in.size()[1], x_in.size()[2]))
    for i in range(x_in.size()[0]):
        x_scaled[i] = torch.tensor(scaler.transform(x_in[i])) 
    return x_scaled, y_in, scaler

### With training

In [ ]:
dict = load_data(12)

In [ ]:
label = "gender"
tensor = "one_hot_tensor_gender"
nb_labels = 3


X_train, y_train, scaler = reshape_for_classif(dict["train"]["X_audio_hubert"], dict["train"]["one_hot_tensor_gender"])
X_test, y_test, _ = reshape_for_classif(dict["test"]["X_audio_hubert"], dict["test"]["one_hot_tensor_gender"], scaler)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

classifier = ConvClassifier(1025, nb_labels)
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

model = train_model(classifier, criterion, optimizer, train_loader, test_loader, num_epochs=100)

### Without training

In [ ]:
dict = load_data(12)

In [ ]:
label = "gender"
tensor = "one_hot_tensor_gender"
nb_labels = 3

X_train, y_train, scaler = reshape_for_classif(dict["train"]["X_audio_hubert"], dict["train"]["one_hot_tensor_gender"])
X_test, y_test, _ = reshape_for_classif(dict["test"]["X_audio_hubert"], dict["test"]["one_hot_tensor_gender"], scaler)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

classifier = ConvClassifier(1025, nb_labels)
classifier.load_state_dict(torch.load("saved_models/gender_audio_classifier.pt"))
optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

with torch.no_grad():
        classifier.eval()
        test_loss, test_accuracy, test_f1, all_labels, all_predictions = test_model(classifier, test_loader, criterion)
        print("Accuracy: ", test_accuracy)
